In [1]:
#Import Libraries
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import norm
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.model_selection import KFold
from IPython.display import HTML, display
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pickle

In [5]:
card_spend = pd.read_csv('CARD_PROCESSED.txt', sep = ',')

In [6]:
card_spend_copy = card_spend.copy()

In [7]:
a=[str(i) for i in card_spend_copy.DONG_CD];
b=[str(i) for i in card_spend_copy.GU_CD];
GUDONG = ["{}{}".format(b_, a_) for a_, b_ in zip(a, b)]
GUDONG = [ int('11'+i+'00') for i in GUDONG]
card_spend_copy['GUDONG'] = GUDONG; card_spend_copy.head()

,STD_DD,GU_CD,DONG_CD,MCT_CAT_CD,SEX_CD,AGE_CD,USE_CNT,USE_AMT,day,month,season,use_per_cnt,use_per_cnt_log,market_name,week,day_label,flow,GUDONG
0,20180401,110,515,21,F,30,4,180,1,4,1,45.000000,3.806662,레저업소,1,1,1356.080,1111051500
1,20180401,110,515,21,F,55,4,22,1,4,1,5.500000,1.704748,레저업소,1,1,1433.060,1111051500
2,20180401,110,515,21,M,20,35,184,1,4,1,5.257143,1.659588,레저업소,1,1,1112.772,1111051500
3,20180401,110,515,21,M,25,70,425,1,4,1,6.071429,1.803594,레저업소,1,1,1792.300,1111051500
4,20180401,110,515,21,M,30,18,82,1,4,1,4.555556,1.516347,레저업소,1,1,1936.100,1111051500


In [8]:
card_spend_copy['flow_level'] = np.trunc(np.log(card_spend_copy.flow))
card_spend_copy['flow_level'] = card_spend_copy.flow_level.astype('int64')

In [9]:
market_name = dict({10:'숙박', 20:'레저용품',21: '레저업소', 22:'문화취미', 30:'가구', 31:'전기', 
                    32:'주방용구', 33:'연료판매', 34:'광학제품', 35: '가전', 40:'유통업',42: '의복',
                    43: '작물', 44: '신변잡화', 50: '서적문구',52: '사무통신',60: '자동차판매',
                    62: '자동차정비', 70:'의료기관', 71:'보건위생', 80:'요식업소', 81: '음료식품', 
                    92: '수리서비스'})

In [10]:
market_names = list(market_name.values());
market_index = list(market_name.keys());

In [11]:
age_index = card_spend_copy.AGE_CD.unique()
age_index.sort(); age_index

array([20, 25, 30, 35, 40, 45, 50, 55, 60, 65], dtype=int64)

In [12]:
sex_ = ['F', 'M']

In [13]:
#read ordernumpy
with open("ordernumpy.pickle", "rb") as f : 
    onp = pickle.load(f)
#read ddict    
with open("ddict.pickle", "rb") as f : 
    ddict = pickle.load(f)

In [14]:
dates = list(ddict.keys())

In [15]:
dates.sort()

In [16]:
# Make Date to Index Dict 
date2idx = {j : i for i, j in enumerate(dates)}

In [17]:
date2idx[20190331]

364

In [18]:
onp[0]

array([10,  4,  7,  3,  8,  6,  1,  6,  3,  1,  9,  8, 10,  6,  6])

In [19]:
def get_onp_row(date_int, i):
    return onp[date2idx[date_int]][i]

In [20]:
# Add ONP rows to corresponding STD_DD Dataframe 
for i in range(15) :
    col_name = "TD_{0}".format(i)
    card_spend_copy[col_name] = [get_onp_row(k, i) for k in card_spend_copy["STD_DD"]]

In [21]:
card_spend_copy.head()

,STD_DD,GU_CD,DONG_CD,MCT_CAT_CD,SEX_CD,AGE_CD,USE_CNT,USE_AMT,day,month,...,TD_5,TD_6,TD_7,TD_8,TD_9,TD_10,TD_11,TD_12,TD_13,TD_14
0,20180401,110,515,21,F,30,4,180,1,4,...,6,1,6,3,1,9,8,10,6,6
1,20180401,110,515,21,F,55,4,22,1,4,...,6,1,6,3,1,9,8,10,6,6
2,20180401,110,515,21,M,20,35,184,1,4,...,6,1,6,3,1,9,8,10,6,6
3,20180401,110,515,21,M,25,70,425,1,4,...,6,1,6,3,1,9,8,10,6,6
4,20180401,110,515,21,M,30,18,82,1,4,...,6,1,6,3,1,9,8,10,6,6


In [22]:
import datetime
def get_date(integer) : 
    year = integer // 10**4
    integer = integer % 10**4
    month = integer // 10**2
    day = integer % 10**2
    return datetime.datetime(year, month, day)

In [23]:
def deter_week(integer):
    integer = get_date(integer).weekday()
    if integer == 5 or integer == 6 : 
        return True
    return False

In [24]:
# Maks masks 
weekmask = card_spend_copy["STD_DD"].map(deter_week) 

In [25]:
# Split 
weekdf, weekenddf = card_spend_copy[~weekmask], card_spend_copy[weekmask]

In [26]:
len(set(weekdf["STD_DD"]))

260

In [27]:
len(set(weekenddf["STD_DD"]))

105

In [28]:
365 / 7 * 5

260.7142857142857

In [29]:
weekdf.to_csv("Week_Data.csv", index = False)

In [30]:
weekenddf.to_csv("WeekEnd_Data.csv", index = False)